### Importing Dataset

In [1]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


ChatGPT was used to learn how to implement the map function.

In [2]:
def mapping(dataset):
    count = {
        1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five',
        6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten',
        11: 'Eleven', 12: 'Twelve'
    }
    dataset['count'] = dataset['Day'].map(count)
    return dataset['count']

In [3]:
xmas.fillna('')

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,,,in a pear tree
1,2,second,dove,,turtle,
2,3,third,hen,,french,
3,4,fourth,bird,,calling,
4,5,fifth,ring,,golden,
5,6,sixth,goose,a-laying,,
6,7,seventh,swan,a-swimming,,
7,8,eighth,maid,a-milking,,
8,9,ninth,lady,dancing,,
9,10,tenth,lord,a-leaping,,


## Function 1

In the following code I am pluralizing the gift depending on the day.

In [4]:
def pluralizeGift(gift):
  
  """
  Returns plural of a noun
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version
  """
  if isinstance(gift, pd.Series):
        return gift.apply(pluralizeGift)
  if 'oo' in gift:
    gift = gift.replace('oo', 'ee')
  elif gift[-1] == 'y':
    gift = gift[:-1] + 'ies'
  else:
    gift = gift + 's'
  return gift

Tesiting if the pluralizeGift Function works
<br>  vectorized pluralizeGift()

In [5]:
# Should work
print(pluralizeGift("goose"))

# Will work if your function is vectorized! 
pluralizeGift(xmas['Gift.Item'])

geese


0     partridges
1          doves
2           hens
3          birds
4          rings
5          geese
6          swans
7          maids
8         ladies
9          lords
10        pipers
11      drummers
Name: Gift.Item, dtype: object

### Function for removing extra whitespaces
**ChatGPT was used her to find the syntax**

In [6]:
def clean_sentence(sentence):
    # Split the sentence into words and join back with a single space to remove extra whitespace
    return ' '.join(sentence.split())

## Function 2

Making the the phrase using inputs

In [7]:
def make_phrase(num, num_word, item, verb, adjective, location):
  """
 Returns the phrase
  
  Parameters
  ----------  
  num : int [ Number of days ]
  num_word: str [ Number of days in words ] 
  item : str [ A noun ]
  verb : str
  ajective : str
  location: str

  Return
  ------
  str
   the phrase
  """
  
  ## Step 1: Replace NAs with blank strings
  verb = verb if verb else ""
  adjective = adjective if adjective else ""
  location = location if location else ""
  
  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  
  ### Hint: call the function you created above!
  if num > 1:
    if item[-1] != 's':
      item = pluralizeGift(item)
  
  ## Step 3: Figure out if a gift item starts with a vowel
  def startVowel(word):
    if word[0].lower() not in ['a','e','i','o','u']:
      return True
  
  ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
  if num == 1:
        article = "an" if startVowel(item) else "a"
        phrase = f"{article} {adjective} {item} {verb} {location}".strip()
  else:
        phrase = f"{num_word} {adjective} {item} {verb} {location}".strip()
  
  ## Step 5: Put all of the pieces together into one string and return!
  return clean_sentence(phrase)


In [8]:
make_phrase(num = 10,
            num_word = "ten", 
            item = "lords", 
            verb = "a-leaping", 
            adjective = "", 
            location = "")

'ten lords a-leaping'

## Function 3

**Used ChatGPT for debugging and figuring out the syntax**

In [9]:
def singDay(dataset , dayNum , phrase_col):
    mapping(dataset)
    day = dataset.loc[dataset['Day'] == dayNum, 'Day.in.Words'].values[0]
    print(f"On the {day} day of Christmas, my true love gave to me:")
    for i in range(dayNum,0,-1):
        gift_row = dataset.loc[dataset['Day'] == i].iloc[0]
        countInWords = gift_row['count']
        adjective = gift_row['Adjective'] if pd.notna(gift_row['Adjective']) else ''
        verb = gift_row['Verb'] if pd.notna(gift_row['Verb']) else ''
        gift_item = pluralizeGift(gift_row['Gift.Item']) if i > 1  else gift_row['Gift.Item']
        location = gift_row['Location'] if pd.notna(gift_row['Location']) else ''
        
        sentence = f"{countInWords} {adjective} {gift_item} {verb} {location}".strip()
        clean_output = clean_sentence(sentence)

        if i == 1 and dayNum > 1: 
            sentence = f"{adjective} {gift_item} {verb} {location}".strip()
            clean_output = clean_sentence(sentence) # Special case for the "partridge in a pear tree"
            print(f"and a {clean_output}.")
        elif i == 1 and dayNum == 1:
            print(f"{clean_output}.")
        else:
            print(f"{clean_output},")
        dataset['phrase_col'] = clean_output

**Testing on xmas dataset**

In [10]:
singDay(xmas,12,'Full.Phrase')

On the twelfth day of Christmas, my true love gave to me:
Twelve drummers drumming,
Eleven pipers piping,
Ten lords a-leaping,
Nine ladies dancing,
Eight maids a-milking,
Seven swans a-swimming,
Six geese a-laying,
Five golden rings,
Four calling birds,
Three french hens,
Two turtle doves,
and a partridge in a pear tree.


**Importing xmas2 dataset**

In [11]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


**Testing singDay function on xmas2 dataset**

In [12]:
singDay(xmas2 , 12,'Full.phrase')

On the twelfth day of Christmas, my true love gave to me:
Twelve hours sleeping,
Eleven friends goodbye-ing,
Ten loads of laundry,
Nine parties bumping,
Eight moms a-calling,
Seven seniors stressing,
Six graders grading,
Five practice exams,
Four course reviews,
Three lost pens,
Two meal points,
and a email from Cal Poly.
